In [1]:
import jax

In [2]:
import anndata as ad
import pandas as pd
import scanpy as sc
import scvi


/home/albert.baichorov/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [3]:
adata = ad.read_h5ad("/home/albert.baichorov/ImSysAging/data/scRNA-seq.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 538266 × 22091
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.HB', 'percent.RPS', 'S.Score', 'G2M.Score', 'Phase', 'umis_TCR', 'umis_BCR', 'primary_type', 'secondary_type', 'sequential_group', 'eight_group', 'sampleName', 'age'
    uns: 'version'
    layers: 'RNA_counts', 'RNA_data'


In [4]:
del adata.layers["RNA_data"]


# Column Descriptions in Annotated Data (AnnData.obs)

This dataset represents the results of single-cell RNA sequencing (scRNA-seq) with cell annotations. Below is a description of the columns:

- **`orig.ident`** – Identifier of the original sample or dataset from which the cell was obtained (e.g., `seurat_object`).
- **`nCount_RNA`** – Total count of RNA molecules (UMIs) detected per cell. Indicates the expression level.
- **`nFeature_RNA`** – Number of unique genes detected in each cell. Reflects transcriptional diversity.
- **`percent.mt`** – Percentage of mitochondrial genes in the total RNA count. High values may indicate stressed or dying cells.
- **`percent.HB`** – Percentage of hemoglobin-related genes in the total RNA count. Useful for analyzing blood cells.
- **`percent.RPS`** – Percentage of ribosomal protein S (RPS) genes in the total RNA count. High values may indicate increased protein biosynthesis.
- **`S.Score`** – Score representing the cell cycle **S-phase** (DNA synthesis phase). Positive values suggest active division.
- **`G2M.Score`** – Score representing the **G2/M phase** of the cell cycle (preparation for mitosis).
- **`Phase`** – Assigned cell cycle phase: $G1$, $S$, or $G2M$.
- **`umis_TCR`** – Number of UMIs mapped to T-cell receptor (TCR) genes. Used to identify T-cell populations.
- **`umis_BCR`** – Number of UMIs mapped to B-cell receptor (BCR) genes. Used to identify B-cell populations.
- **`primary_type`** – Broad classification of the cell type (e.g., CD4+ T cells, CD8+ T cells, B cells).
- **`secondary_type`** – More detailed subtype of the cell, such as `CD4_TEM_ANXA1` (effector memory CD4+ T cells expressing ANXA1) or `B_Memory` (memory B cells).
- **`sequential_group`** – Categorical age group label, such as `60y`.
- **`eight_group`** – Alternative age grouping format, such as `50-60y`.
- **`sampleName`** – Sample or dataset identifier (e.g., `sample42`), indicating the source of the cell.
- **`age`** – Age of the donor in years (e.g., `60`).

The data is stored in the AnnData format and can be used for analyzing cellular populations, identifying cell states, and comparing different age groups.


In [7]:
raw_counts = adata.layers["RNA_counts"]

df_counts = pd.DataFrame(
    raw_counts[:10, ].toarray(),  # Convert to dense
    index=adata.obs_names[:10],  # Cell barcodes
    columns=adata.var_names  # Gene names
)

df_counts.head()


,AL627309.1,AL669831.5,LINC00115,AL645608.3,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,AL645608.2,...,AC084398.2,OR4E1,PRR35,AC018553.2,AC005180.1,AC025048.1,AC015802.1,AP005137.2,AL121917.2,AC016590.3
AAACCTGAGGCGTACA-1-sample42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGGGAGTAA-1-sample42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGCAGCGATCC-1-sample42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGCATATGGTC-1-sample42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGCATTGAGCT-1-sample42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
from scvi.external import MRVI
import anndata as ad
import torch

import jax
print(jax.devices())


adata = ad.read_h5ad("/home/albert.baichorov/ImSysAging/data/scRNA-seq.h5ad")

/home/albert.baichorov/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


[CpuDevice(id=0)]
